In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import make_blobs
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)


In [10]:
def my_softmax(z):
    ez = np.exp(z)              #element-wise exponenial
    sm = ez/np.sum(ez)
    return(sm)

In [11]:
# make  dataset for example
centers = [[-3, 3], [-2, -2], [1, 2], [5, -2]]
X_train, y_train = make_blobs(n_samples=2000, centers=centers, cluster_std=1.0,random_state=30)

### Less Accurate Method
This method yields less numerically accurate method, a better approach is to use logistic with the last output layer, a linear layer

In [12]:
model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'softmax')    # < softmax activation here
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X_train,y_train,
    epochs=10
)
     

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 376us/step - loss: 1.6226
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 346us/step - loss: 0.7266
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step - loss: 0.4932
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step - loss: 0.2756
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 320us/step - loss: 0.1483
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/step - loss: 0.1078
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 329us/step - loss: 0.1006
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step - loss: 0.0902
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 384us/step - loss: 0.0830
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 331us/step - loss: 0.0709


In [13]:
value_first = model.predict(X_train)
print(value_first [:2])
print("largest value", np.max(value_first), "smallest value", np.min(value_first))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step
[[7.3915720e-03 2.8557498e-03 9.6399033e-01 2.5762375e-02]
 [9.8848695e-01 9.5212692e-03 1.5579092e-03 4.3385942e-04]]
largest value 0.99999976 smallest value 5.1562106e-09


### Yields better result
It uses a linear activation function for the output layer

In [15]:
preferred_model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')  
    ]
)
preferred_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

preferred_model.fit(
    X_train,y_train,
    epochs=10
)
       

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step - loss: 0.9240
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - loss: 0.3810
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step - loss: 0.1859
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 333us/step - loss: 0.1187
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 357us/step - loss: 0.0880
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 337us/step - loss: 0.0694
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 342us/step - loss: 0.0893
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step - loss: 0.0703
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - loss: 0.0631
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step - loss: 0.0592


In [16]:
value_second = preferred_model.predict(X_train)
print(f"two example output vectors:\n {value_second[:2]}")
print("largest value", np.max(value_second), "smallest value", np.min(value_second))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step
two example output vectors:
 [[-1.8737134 -1.2479361  3.3634593 -1.119853 ]
 [ 4.3558846 -1.3363942 -3.2306595 -8.346227 ]]
largest value 10.679079 smallest value -16.833454


Now converting these values into probabilities

In [18]:
sm_preferred = tf.nn.softmax(value_second).numpy()
print(f"two example output vectors:\n {sm_preferred[:2]}")
print("largest value", np.max(sm_preferred), "smallest value", np.min(sm_preferred))

two example output vectors:
 [[5.1777982e-03 9.6809221e-03 9.7413749e-01 1.1003791e-02]
 [9.9613273e-01 3.3588600e-03 5.0526939e-04 3.0329163e-06]]
largest value 0.99999976 smallest value 1.2623603e-12
